# High-Level Summary

## 1. Clipping

- Parse `Labels-v2.json` → extract second-half goal times.
- Cut 7-second clips around goals (±3 seconds) → save to `Clips/goal/`.
- Generate the same number of random 7-second non-goal clips, avoiding events → save to `Clips/no goal/`.

**Output after this step**: Two folders of raw 7-second video clips:
- One containing goals.
- One containing matched-duration “negative” clips.

---

## 2. Frame-Level Preprocessing

Using OpenCV to clean each 7-second clip so that models focus on the ball movement, not distracting background:

1. Read each clip frame-by-frame, convert to grayscale.
2. Detect and mask grass via HSV thresholding (green range), and mask audience (another HSV range).
3. Detect the ball by combining:
   - HSV threshold for bright (white) regions.
   - High-intensity pixels in the gray frame.
   - Then clean up via morphology.
4. Combine masks so that only the ball remains “unmasked,” everything else (grass, crowd) is blacked out.

**Final Processed Videos**:
- Saved in `Goal p1` and `NoGoal p1`.

In [ ]:
import os
import json
import random
import subprocess
from pathlib import Path
from typing import List, Tuple, Dict, Any

class FootballClipGenerator:
    def __init__(self, workspace_path: str):
        """
        Initialize the clip generator with workspace path
        
        Args:
            workspace_path: Path to the AIM Lab/Experiment directory
        """
        self.workspace_path = Path(workspace_path)
        self.clips_dir = self.workspace_path / "Clips"
        self.goal_dir = self.clips_dir / "goal"
        self.no_goal_dir = self.clips_dir / "no goal"
        
        # Create clips directories if they don't exist
        self.goal_dir.mkdir(parents=True, exist_ok=True)
        self.no_goal_dir.mkdir(parents=True, exist_ok=True)
        
        # Events that should be avoided when generating random clips
        self.avoided_events = [
            "Goal", "Penalty", "Free kick", "Foul", 
            "Red card", "Yellow card", "Corner", "Offside"
        ]
    
    def parse_game_time(self, game_time: str) -> Tuple[int, int]:
        """
        Parse game time format "2 - 48:33" to get half and seconds
        
        Args:
            game_time: Time string in format "half - mm:ss"
            
        Returns:
            Tuple of (half, total_seconds_in_half)
        """
        try:
            half_str, time_str = game_time.split(" - ")
            half = int(half_str)
            
            minutes, seconds = map(int, time_str.split(":"))
            total_seconds = minutes * 60 + seconds
            
            return half, total_seconds
        except (ValueError, IndexError):
            return 0, 0
    
    def get_video_duration(self, video_path: Path) -> float:
        """
        Get video duration using ffprobe
        
        Args:
            video_path: Path to the video file
            
        Returns:
            Duration in seconds
        """
        try:
            cmd = [
                'ffprobe', '-v', 'quiet', '-show_entries', 'format=duration',
                '-of', 'default=noprint_wrappers=1:nokey=1', str(video_path)
            ]
            result = subprocess.run(cmd, capture_output=True, text=True)
            return float(result.stdout.strip())
        except:
            return 0.0
    
    def extract_clip(self, video_path: Path, start_time: float, duration: float, output_path: Path) -> bool:
        """
        Extract video clip using ffmpeg
        
        Args:
            video_path: Source video path
            start_time: Start time in seconds
            duration: Duration in seconds
            output_path: Output clip path
            
        Returns:
            True if successful, False otherwise
        """
        try:
            cmd = [
                'ffmpeg', '-i', str(video_path),
                '-ss', str(start_time),
                '-t', str(duration),
                '-c', 'copy',
                '-avoid_negative_ts', 'make_zero',
                str(output_path),
                '-y'  # Overwrite output file
            ]
            result = subprocess.run(cmd, capture_output=True, text=True)
            return result.returncode == 0
        except:
            return False
    
    def process_labels_file(self, labels_path: Path) -> Tuple[List[Dict], List[Tuple[float, float]]]:
        """
        Process Labels-v2.json file to extract goals and event times
        
        Args:
            labels_path: Path to Labels-v2.json file
            
        Returns:
            Tuple of (second_half_goals, all_event_intervals)
        """
        try:
            with open(labels_path, 'r') as f:
                data = json.load(f)
            
            second_half_goals = []
            all_event_intervals = []
            
            for annotation in data.get('annotations', []):
                game_time = annotation.get('gameTime', '')
                label = annotation.get('label', '')
                
                half, seconds = self.parse_game_time(game_time)
                
                if half == 2:  # Second half only
                    if label == "Goal":
                        second_half_goals.append({
                            'time': seconds,
                            'team': annotation.get('team', ''),
                            'annotation': annotation
                        })
                    
                    # Add all events to avoid list (with buffer)
                    if label in self.avoided_events:
                        start_avoid = max(0, seconds - 5)  # 5 second buffer before
                        end_avoid = seconds + 5  # 5 second buffer after
                        all_event_intervals.append((start_avoid, end_avoid))
            
            return second_half_goals, all_event_intervals
            
        except Exception as e:
            print(f"Error processing {labels_path}: {e}")
            return [], []
    
    def generate_random_segments(self, video_duration: float, avoided_intervals: List[Tuple[float, float]], 
                                num_segments: int, segment_duration: float = 7.0) -> List[float]:
        """
        Generate random 7-second segments avoiding event intervals
        
        Args:
            video_duration: Total video duration
            avoided_intervals: List of (start, end) intervals to avoid
            num_segments: Number of segments to generate
            segment_duration: Duration of each segment
            
        Returns:
            List of start times for random segments
        """
        random_segments = []
        max_attempts = 1000
        
        for _ in range(num_segments):
            attempts = 0
            while attempts < max_attempts:
                # Generate random start time (ensuring we don't go beyond video)
                max_start = video_duration - segment_duration
                if max_start <= 0:
                    break
                
                start_time = random.uniform(0, max_start)
                end_time = start_time + segment_duration
                
                # Check if this segment overlaps with any avoided interval
                overlap = False
                for avoid_start, avoid_end in avoided_intervals:
                    if not (end_time <= avoid_start or start_time >= avoid_end):
                        overlap = True
                        break
                
                if not overlap:
                    random_segments.append(start_time)
                    break
                
                attempts += 1
            
            if attempts >= max_attempts:
                print(f"Warning: Could not find non-overlapping segment after {max_attempts} attempts")
        
        return random_segments
    
    def process_match_folder(self, match_folder: Path) -> Dict[str, Any]:
        """
        Process a single match folder
        
        Args:
            match_folder: Path to match folder containing video and labels
            
        Returns:
            Dictionary with processing results
        """
        video_path = match_folder / "2_720p.mkv"
        labels_path = match_folder / "Labels-v2.json"
        
        result = {
            'match_folder': match_folder.name,
            'goals_found': 0,
            'clips_generated': 0,
            'success': False
        }
        
        # Check if required files exist
        if not video_path.exists():
            print(f"Video file not found: {video_path}")
            return result
        
        if not labels_path.exists():
            print(f"Labels file not found: {labels_path}")
            return result
        
        # Process labels file
        second_half_goals, event_intervals = self.process_labels_file(labels_path)
        result['goals_found'] = len(second_half_goals)
        
        # If no goals in second half, skip this match
        if not second_half_goals:
            print(f"No goals in second half for {match_folder.name}")
            return result
        
        # Get video duration
        video_duration = self.get_video_duration(video_path)
        if video_duration == 0:
            print(f"Could not get video duration for {video_path}")
            return result
        
        print(f"Processing {match_folder.name} - {len(second_half_goals)} goals found")
        
        # Generate goal clips
        goal_clips_generated = 0
        for i, goal in enumerate(second_half_goals, 1):
            goal_time = goal['time']
            start_time = max(0, goal_time - 3)  # 3 seconds before
            end_time = min(video_duration, goal_time + 3)  # 3 seconds after
            clip_duration = end_time - start_time
            
            if clip_duration > 0:
                goal_clip_path = self.goal_dir / f"g{goal_clips_generated + 1}_{match_folder.name}.mp4"
                
                if self.extract_clip(video_path, start_time, clip_duration, goal_clip_path):
                    goal_clips_generated += 1
                    print(f"Generated goal clip: {goal_clip_path.name}")
                else:
                    print(f"Failed to generate goal clip for goal at {goal_time}s")
        
        # Generate random non-goal clips
        random_start_times = self.generate_random_segments(
            video_duration, event_intervals, len(second_half_goals), 7.0
        )
        
        non_goal_clips_generated = 0
        for i, start_time in enumerate(random_start_times):
            non_goal_clip_path = self.no_goal_dir / f"ng{non_goal_clips_generated + 1}_{match_folder.name}.mp4"
            
            if self.extract_clip(video_path, start_time, 7.0, non_goal_clip_path):
                non_goal_clips_generated += 1
                print(f"Generated non-goal clip: {non_goal_clip_path.name}")
            else:
                print(f"Failed to generate non-goal clip starting at {start_time}s")
        
        result['clips_generated'] = goal_clips_generated + non_goal_clips_generated
        result['success'] = True
        
        return result
    
    def process_all_matches(self) -> Dict[str, Any]:
        """
        Process all match folders in the workspace
        
        Returns:
            Summary of processing results
        """
        summary = {
            'total_matches': 0,
            'matches_processed': 0,
            'total_goals': 0,
            'total_clips': 0,
            'matches_with_goals': 0
        }
        
        # Process both season folders
        season_folders = [
            self.workspace_path / "2014-2015",
            self.workspace_path / "2015-2014"
        ]
        
        for season_folder in season_folders:
            if not season_folder.exists():
                print(f"Season folder not found: {season_folder}")
                continue
            
            print(f"\nProcessing season: {season_folder.name}")
            
            # Iterate through all match folders in the season
            for match_folder in season_folder.iterdir():
                if match_folder.is_dir():
                    summary['total_matches'] += 1
                    
                    result = self.process_match_folder(match_folder)
                    
                    if result['success']:
                        summary['matches_processed'] += 1
                        summary['total_goals'] += result['goals_found']
                        summary['total_clips'] += result['clips_generated']
                        
                        if result['goals_found'] > 0:
                            summary['matches_with_goals'] += 1
        
        return summary
    
    def run(self):
        """
        Main execution method
        """
        print("Football Clip Generator Starting...")
        print(f"Workspace: {self.workspace_path}")
        print(f"Goal clips will be saved to: {self.goal_dir}")
        print(f"Non-goal clips will be saved to: {self.no_goal_dir}")
        
        # Clean existing clips (optional)
        response = input("Do you want to clean existing clips? (y/n): ").lower()
        if response == 'y':
            for clip_file in self.goal_dir.glob("*.mp4"):
                clip_file.unlink()
            for clip_file in self.no_goal_dir.glob("*.mp4"):
                clip_file.unlink()
            print("Existing clips cleaned.")
        
        # Process all matches
        summary = self.process_all_matches()
        
        # Print summary
        print("\n" + "="*50)
        print("PROCESSING SUMMARY")
        print("="*50)
        print(f"Total matches found: {summary['total_matches']}")
        print(f"Matches processed successfully: {summary['matches_processed']}")
        print(f"Matches with goals in 2nd half: {summary['matches_with_goals']}")
        print(f"Total goals found: {summary['total_goals']}")
        print(f"Total clips generated: {summary['total_clips']}")
        print(f"Goal clips saved to: {self.goal_dir}")
        print(f"Non-goal clips saved to: {self.no_goal_dir}")


def main():
    """
    Main function to run the clip generator
    """
    # Set your workspace path here
    workspace_path = r"F:\AIM Lab\Experiment"  # Adjust this path as needed
    
    # Check if workspace exists
    if not os.path.exists(workspace_path):
        print(f"Workspace path not found: {workspace_path}")
        print("Please update the workspace_path variable with the correct path.")
        return
    
    # Create and run the clip generator
    generator = FootballClipGenerator(workspace_path)
    generator.run()


if __name__ == "__main__":
    main()

In [16]:
import os
import json
from moviepy import VideoFileClip
import warnings
warnings.filterwarnings('ignore')

def extract_goal_clips_moviepy():
    """
    Iterate through match folders, extract goal timings from Labels-v2.json,
    and create 7-second clips around goals in the second half using MoviePy.
    """
    
    # Base directory containing season folders
    base_dir = "."
    clips_dir = os.path.join(base_dir, "Clips", "goal")
    
    # Create clips directory if it doesn't exist
    os.makedirs(clips_dir, exist_ok=True)
    
    # Counter for goal clips
    goal_counter = 1
    
    # Iterate through season folders (e.g., 2014-2015, 2015-2014)
    for season_folder in os.listdir(base_dir):
        season_path = os.path.join(base_dir, season_folder)
        if os.path.isdir(season_path) and season_folder.startswith("20"):
            print(f"Processing season: {season_folder}")
            
            # Iterate through match folders
            for match_folder in os.listdir(season_path):
                match_path = os.path.join(season_path, match_folder)
                if os.path.isdir(match_path):
                    print(f"  Processing match: {match_folder}")
                    
                    # Look for Labels-v2.json file
                    labels_file = os.path.join(match_path, "Labels-v2.json")
                    video_file = os.path.join(match_path, "2_720p.mkv")
                    
                    if os.path.exists(labels_file) and os.path.exists(video_file):
                        # Extract goals from second half
                        second_half_goals = extract_second_half_goals(labels_file)
                        
                        if second_half_goals:
                            print(f"    Found {len(second_half_goals)} goal(s) in second half")
                            
                            # Load video once for all clips from this match
                            try:
                                video = VideoFileClip(video_file)
                                
                                # Create clips for each goal
                                for goal in second_half_goals:
                                    create_goal_clip_moviepy(
                                        video, 
                                        goal['position'], 
                                        clips_dir, 
                                        f"g{goal_counter}",
                                        goal['gameTime']
                                    )
                                    goal_counter += 1
                                
                                # Close video to free memory
                                video.close()
                                
                            except Exception as e:
                                print(f"    Error loading video: {e}")
                        else:
                            print(f"    No goals found in second half")
                    else:
                        if not os.path.exists(labels_file):
                            print(f"    Labels-v2.json not found")
                        if not os.path.exists(video_file):
                            print(f"    2_720p.mkv not found")

def extract_second_half_goals(labels_file):
    """
    Extract goal events from the second half of the match.
    
    Args:
        labels_file (str): Path to the Labels-v2.json file
        
    Returns:
        list: List of goal events in second half with position and gameTime
    """
    try:
        with open(labels_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        second_half_goals = []
        
        for annotation in data.get('annotations', []):
            # Check if it's a goal in the second half
            if (annotation.get('label') == 'Goal' and 
                annotation.get('gameTime', '').startswith('2 -')):
                
                second_half_goals.append({
                    'position': int(annotation.get('position', 0)),
                    'gameTime': annotation.get('gameTime', ''),
                    'team': annotation.get('team', 'unknown')
                })
        
        return second_half_goals
        
    except Exception as e:
        print(f"    Error reading labels file: {e}")
        return []

def create_goal_clip_moviepy(video_clip, position_ms, output_dir, clip_name, game_time):
    """
    Create a 7-second video clip around the goal time using MoviePy.
    
    Args:
        video_clip (VideoFileClip): Loaded video clip
        position_ms (int): Goal position in milliseconds
        output_dir (str): Directory to save the clip
        clip_name (str): Name for the output clip (e.g., 'g1')
        game_time (str): Game time for logging
    """
    try:
        # Calculate start and end times (3 seconds before and after)
        start_seconds = max(0, (position_ms - 3000) / 1000.0)  # 3 seconds before, but not negative
        end_seconds = (position_ms + 4000) / 1000.0  # 4 seconds after (total 7 seconds)
        
        # Make sure we don't exceed video duration
        end_seconds = min(end_seconds, video_clip.duration)
        
        # Output file path
        output_file = os.path.join(output_dir, f"{clip_name}.mp4")
        
        print(f"    Creating clip {clip_name} for goal at {game_time}")
        print(f"    Start: {start_seconds:.2f}s, End: {end_seconds:.2f}s")
        
        # Extract the clip
        goal_clip = video_clip.subclipped(start_seconds, end_seconds)
        
        # Write the clip to file
        goal_clip.write_videofile(
            output_file, 
            audio_codec='aac',
            codec='libx264',
            logger=None  # Suppress moviepy logs
        )
        
        # Close the clip to free memory
        goal_clip.close()
        
        print(f"    Successfully created {output_file}")
            
    except Exception as e:
        print(f"    Error creating clip {clip_name}: {e}")

def extract_goal_clips_moviepy():
    """
    Iterate through match folders, extract goal timings from Labels-v2.json,
    and create 7-second clips around goals in the second half using MoviePy.
    Additionally, generate random clips of the same size as the goal clips.
    """
    
    # Base directory containing season folders
    base_dir = "."
    clips_dir = os.path.join(base_dir, "Clips", "goal")
    random_clips_dir = os.path.join(base_dir, "Clips", "no goal")
    
    # Create clips directories if they don't exist
    os.makedirs(clips_dir, exist_ok=True)
    os.makedirs(random_clips_dir, exist_ok=True)
    
    # Counter for goal and random clips
    goal_counter = 1
    random_counter = 1
    
    # Iterate through season folders (e.g., 2014-2015, 2015-2014)
    for season_folder in os.listdir(base_dir):
        season_path = os.path.join(base_dir, season_folder)
        if os.path.isdir(season_path) and season_folder.startswith("20"):
            print(f"Processing season: {season_folder}")
            
            # Iterate through match folders
            for match_folder in os.listdir(season_path):
                match_path = os.path.join(season_path, match_folder)
                if os.path.isdir(match_path):
                    print(f"  Processing match: {match_folder}")
                    
                    # Look for Labels-v2.json file
                    labels_file = os.path.join(match_path, "Labels-v2.json")
                    video_file = os.path.join(match_path, "2_720p.mkv")
                    
                    if os.path.exists(labels_file) and os.path.exists(video_file):
                        # Extract goals from second half
                        second_half_goals = extract_second_half_goals(labels_file)
                        
                        if second_half_goals:
                            print(f"    Found {len(second_half_goals)} goal(s) in second half")
                            
                            # Load video once for all clips from this match
                            try:
                                video = VideoFileClip(video_file)
                                video_duration = video.duration
                                
                                # Create clips for each goal
                                for goal in second_half_goals:
                                    create_goal_clip_moviepy(
                                        video, 
                                        goal['position'], 
                                        clips_dir, 
                                        f"g{goal_counter}",
                                        goal['gameTime']
                                    )
                                    goal_counter += 1
                                
                                # Generate random clips
                                for _ in range(len(second_half_goals)):
                                    random_start_time = generate_random_start_time(video_duration, 6)
                                    create_random_clip_moviepy(
                                        video, 
                                        random_start_time, 
                                        random_clips_dir, 
                                        f"ng{random_counter}"
                                    )
                                    random_counter += 1
                                
                                # Close video to free memory
                                video.close()
                                
                            except Exception as e:
                                print(f"    Error loading video: {e}")
                        else:
                            print(f"    No goals found in second half")
                    else:
                        if not os.path.exists(labels_file):
                            print(f"    Labels-v2.json not found")
                        if not os.path.exists(video_file):
                            print(f"    2_720p.mkv not found")

def extract_second_half_goals(labels_file):
    """
    Extract goal events from the second half of the match.
    
    Args:
        labels_file (str): Path to the Labels-v2.json file
        
    Returns:
        list: List of goal events in second half with position and gameTime
    """
    try:
        with open(labels_file, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        second_half_goals = []
        
        for annotation in data.get('annotations', []):
            # Check if it's a goal in the second half
            if (annotation.get('label') == 'Goal' and 
                annotation.get('gameTime', '').startswith('2 -')):
                
                second_half_goals.append({
                    'position': int(annotation.get('position', 0)),
                    'gameTime': annotation.get('gameTime', ''),
                    'team': annotation.get('team', 'unknown')
                })
        
        return second_half_goals
        
    except Exception as e:
        print(f"    Error reading labels file: {e}")
        return []

def create_goal_clip_moviepy(video_clip, position_ms, output_dir, clip_name, game_time):
    """
    Create a 7-second video clip around the goal time using MoviePy.
    
    Args:
        video_clip (VideoFileClip): Loaded video clip
        position_ms (int): Goal position in milliseconds
        output_dir (str): Directory to save the clip
        clip_name (str): Name for the output clip (e.g., 'g1')
        game_time (str): Game time for logging
    """
    try:
        # Calculate start and end times (3 seconds before and after)
        start_seconds = max(0, (position_ms - 3000) / 1000.0)  # 3 seconds before, but not negative
        end_seconds = (position_ms + 4000) / 1000.0  # 4 seconds after (total 7 seconds)
        
        # Make sure we don't exceed video duration
        end_seconds = min(end_seconds, video_clip.duration)
        
        # Output file path
        output_file = os.path.join(output_dir, f"{clip_name}.mp4")
        
        print(f"    Creating clip {clip_name} for goal at {game_time}")
        print(f"    Start: {start_seconds:.2f}s, End: {end_seconds:.2f}s")
        
        # Extract the clip
        goal_clip = video_clip.subclipped(start_seconds, end_seconds)
        
        # Write the clip to file
        goal_clip.write_videofile(
            output_file, 
            audio_codec='aac',
            codec='libx264',
            logger=None  # Suppress moviepy logs
        )
        
        # Close the clip to free memory
        goal_clip.close()
        
        print(f"    Successfully created {output_file}")
            
    except Exception as e:
        print(f"    Error creating clip {clip_name}: {e}")

def create_random_clip_moviepy(video_clip, start_time, output_dir, clip_name):
    """
    Create a random 6-second video clip.
    
    Args:
        video_clip (VideoFileClip): Loaded video clip
        start_time (float): Start time in seconds
        output_dir (str): Directory to save the clip
        clip_name (str): Name for the output clip (e.g., 'ng1')
    """
    try:
        # Calculate end time
        end_time = start_time + 6.0  # 6 seconds duration
        
        # Make sure we don't exceed video duration
        end_time = min(end_time, video_clip.duration)
        
        # Output file path
        output_file = os.path.join(output_dir, f"{clip_name}.mp4")
        
        print(f"    Creating random clip {clip_name}")
        print(f"    Start: {start_time:.2f}s, End: {end_time:.2f}s")
        
        # Extract the clip
        random_clip = video_clip.subclipped(start_time, end_time)
        
        # Write the clip to file
        random_clip.write_videofile(
            output_file, 
            audio_codec='aac',
            codec='libx264',
            logger=None  # Suppress moviepy logs
        )
        
        # Close the clip to free memory
        random_clip.close()
        
        print(f"    Successfully created {output_file}")
            
    except Exception as e:
        print(f"    Error creating random clip {clip_name}: {e}")

def generate_random_start_time(video_duration, clip_duration):
    """
    Generate a random start time for a clip.
    
    Args:
        video_duration (float): Total duration of the video
        clip_duration (float): Duration of the clip
        
    Returns:
        float: Random start time in seconds
    """
    max_start_time = max(0, video_duration - clip_duration)
    return random.uniform(0, max_start_time)


# Run the extraction
print("Goal Clip Extractor (MoviePy)")
print("=============================")

try:
    extract_goal_clips_moviepy()
    print("\nProcessing complete!")
except ImportError:
    print("Error: MoviePy not installed. Install with: pip install moviepy")
except Exception as e:
    print(f"Error: {e}")

Goal Clip Extractor (MoviePy)
Processing season: 2014-2015
  Processing match: 2015-02-21 - 18-00 Chelsea 1 - 1 Burnley
    Found 1 goal(s) in second half
    Creating clip g1 for goal at 2 - 35:21
    Start: 2118.49s, End: 2125.49s
    Successfully created .\Clips\goal\g1.mp4
    Creating random clip ng1
    Start: 1607.11s, End: 1613.11s
Proc not detected
    Successfully created .\Clips\no goal\ng1.mp4
  Processing match: 2015-02-21 - 18-00 Crystal Palace 1 - 2 Arsenal
    Found 1 goal(s) in second half
    Creating clip g2 for goal at 2 - 48:33
    Start: 2910.95s, End: 2917.95s
    Successfully created .\Clips\goal\g2.mp4
    Creating random clip ng2
    Start: 587.56s, End: 593.56s
Proc not detected
    Successfully created .\Clips\no goal\ng2.mp4
  Processing match: 2015-02-21 - 18-00 Swansea 2 - 1 Manchester United
    Found 1 goal(s) in second half
    Creating clip g3 for goal at 2 - 27:07
    Start: 1624.79s, End: 1631.79s
    Successfully created .\Clips\goal\g3.mp4
    Cre

In [17]:
import os
import cv2
import numpy as np

def process_video(input_path, output_path):
    """
    Process the video to:
    1. Convert to grayscale.
    2. Remove grass and audience areas while preserving the ball.
    """
    # Load video
    cap = cv2.VideoCapture(input_path)
    
    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Changed to mp4v for better compatibility
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height), isColor=False)
    
    print(f"Processing {total_frames} frames from {input_path}...")
    
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1
        if frame_count % 100 == 0:
            print(f"Processed {frame_count}/{total_frames} frames")

        # Convert to grayscale
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Convert original frame to HSV for better color detection
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        # Define green grass color range in HSV (more specific range)
        lower_green = np.array([35, 40, 40])
        upper_green = np.array([85, 255, 255])
        grass_mask = cv2.inRange(hsv, lower_green, upper_green)

        # Define ball detection (white/light colored ball)
        lower_ball_hsv = np.array([0, 0, 200])
        upper_ball_hsv = np.array([180, 30, 255])
        ball_mask_hsv = cv2.inRange(hsv, lower_ball_hsv, upper_ball_hsv)
        
        _, ball_mask_gray = cv2.threshold(gray_frame, 200, 255, cv2.THRESH_BINARY)
        ball_mask = cv2.bitwise_or(ball_mask_hsv, ball_mask_gray)
        
        kernel = np.ones((3, 3), np.uint8)
        ball_mask = cv2.morphologyEx(ball_mask, cv2.MORPH_CLOSE, kernel)
        ball_mask = cv2.morphologyEx(ball_mask, cv2.MORPH_OPEN, kernel)

        lower_audience = np.array([0, 100, 150])
        upper_audience = np.array([180, 255, 255])
        audience_mask = cv2.inRange(hsv, lower_audience, upper_audience)
        audience_mask = cv2.bitwise_and(audience_mask, cv2.bitwise_not(ball_mask))

        combined_mask = cv2.bitwise_or(grass_mask, audience_mask)
        keep_mask = cv2.bitwise_not(combined_mask)
        keep_mask = cv2.bitwise_or(keep_mask, ball_mask)

        processed_frame = cv2.bitwise_and(gray_frame, gray_frame, mask=keep_mask)
        out.write(processed_frame)
    
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Processed video saved as: {output_path}")

def process_all_videos(input_folder, output_folder):
    """
    Process all videos in the input folder and save them to the output folder.
    
    Args:
        input_folder (str): Path to the folder containing input videos.
        output_folder (str): Path to the folder to save processed videos.
    """
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for video_file in os.listdir(input_folder):
        input_path = os.path.join(input_folder, video_file)
        output_path = os.path.join(output_folder, video_file)
        
        if os.path.isfile(input_path) and video_file.endswith(".mp4"):
            process_video(input_path, output_path)

if __name__ == "__main__":
    # Define input and output folders
    base_path = "F:/AIM Lab/Experiment/Clips"
    goal_input_folder = os.path.join(base_path, "goal")
    no_goal_input_folder = os.path.join(base_path, "no goal")
    goal_output_folder = os.path.join(base_path, "Goal p1")
    no_goal_output_folder = os.path.join(base_path, "NoGoal p1")
    
    # Process videos in the "goal" folder
    print("Processing videos in the 'goal' folder...")
    process_all_videos(goal_input_folder, goal_output_folder)
    
    # Process videos in the "no goal" folder
    print("Processing videos in the 'no goal' folder...")
    process_all_videos(no_goal_input_folder, no_goal_output_folder)

Processing videos in the 'goal' folder...
Processing 175 frames from F:/AIM Lab/Experiment/Clips\goal\g1.mp4...
Processed 100/175 frames
Processed video saved as: F:/AIM Lab/Experiment/Clips\Goal p1\g1.mp4
Processing 175 frames from F:/AIM Lab/Experiment/Clips\goal\g10.mp4...
Processed 100/175 frames
Processed video saved as: F:/AIM Lab/Experiment/Clips\Goal p1\g10.mp4
Processing 175 frames from F:/AIM Lab/Experiment/Clips\goal\g11.mp4...
Processed 100/175 frames
Processed video saved as: F:/AIM Lab/Experiment/Clips\Goal p1\g11.mp4
Processing 175 frames from F:/AIM Lab/Experiment/Clips\goal\g12.mp4...
Processed 100/175 frames
Processed video saved as: F:/AIM Lab/Experiment/Clips\Goal p1\g12.mp4
Processing 175 frames from F:/AIM Lab/Experiment/Clips\goal\g13.mp4...
Processed 100/175 frames
Processed video saved as: F:/AIM Lab/Experiment/Clips\Goal p1\g13.mp4
Processing 175 frames from F:/AIM Lab/Experiment/Clips\goal\g14.mp4...
Processed 100/175 frames
Processed video saved as: F:/AIM L